#Sample datasets

https://docs.databricks.com/aws/en/discover/databricks-datasets

##Unity Catalog datasets

In [0]:
%sql
SELECT * FROM samples.nyctaxi.trips LIMIT 10

In [0]:
%sql
SHOW TABLES IN samples.tpch

##Databricks datasets (databricks-datasets) mounted to DBFS

In [0]:
display(dbutils.fs.ls('/databricks-datasets'))

In [0]:
display(dbutils.fs.ls('dbfs:/databricks-datasets/cs190/data-001/'))

In [0]:
dbutils.fs.head('dbfs:/databricks-datasets/iot/iot_devices.json')